In [12]:
## Load data
import glob
import pandas as pd
import pickle

group = 'ageAtEnroll' #race


file_path_df = '../results/preprocessed_data/1_2_model_input_ws60min_ph60min_v6.csv'
df_cgm = pd.read_csv(file_path_df)
df_cgm.drop(columns = ['EduLevel', 'Value_1', 'Value_2','Value_3','Value_4','Target'], inplace=True)
df_cgm = df_cgm.dropna()
df_cgm = df_cgm.drop_duplicates(subset=['PtID'], keep='first')


df_cgm


,PtID,Race,Gender,ageAtEnroll
0,1,black,F,15
8056,2,black,F,15
11886,3,white,M,15
20677,4,black,M,16
30667,5,black,F,43
...,...,...,...,...
1711495,228,white,F,58
1720433,229,white,F,51
1729167,230,black,F,45
1738562,231,white,F,54


In [13]:

# Define the path to your folder
folder_path = rf'/home/hbt/jchr_data/jchr_racial_diff/results/processed_data/2_1_1_predicted_results_rnn_v6_{group}_v2'

# Pattern to match the files
file_pattern = f'{folder_path}/patient*_ratio*.pkl'

# Dictionary to store the data
dictionary = {}

# Iterate over files matched by glob
for file_path in glob.glob(file_pattern):
    with open(file_path, 'rb') as file:
        # Extracting N and X from the filename
        filename = file_path.split('/')[-1]  # Adjust if necessary based on your OS
        patient_id, ratio_id = filename.replace('.pkl', '').split('_')
        # Extracting numeric parts from patient_id and ratio_id
        patient_num = ''.join(filter(str.isdigit, patient_id))
        ratio_num = ''.join(filter(str.isdigit, ratio_id))
        
        # Load the content of the file
        file_data = pickle.load(file)

        # Store the data
        dictionary.update(file_data)

# Now loaded_data contains all your files' data

In [14]:
import my_utils
import numpy as np
iteration = 1
dict_results = {}

for (PtID, percentage), value in dictionary.items():

    # print('Iteration:',iteration,'PtID:',PtID,'ratio:' ,percentage)
    iteration=iteration+1
    
    # My actual/true values and my baseline value
    y_actual = value['y_actual']
    y_last_value = value['y_last_val']
    y_pred_single = value['y_pred_single']
    y_pred_base = value['y_pred']
    y_pred_tl = value['y_pred_tl']
    y_last_value2 = y_last_value.values.reshape(-1, 1) # make y_last_val to nparray (672,1) from Series (672,)
    
    train_size_single_tl = value["train_size_single_tl"]
    train_size_base = value["train_size_base"]
    train_size_naive = value["train_size_naive"]
        
    # calculating my values for each prediction base_model
    rmse_naive = my_utils.calculate_results(y_actual, y_last_value2)
    rmse_single = my_utils.calculate_results(y_actual, y_pred_single)
    rmse_base = my_utils.calculate_results(y_actual, y_pred_base)
    rmse_tl = my_utils.calculate_results(y_actual,y_pred_tl)
    
     
    dict_results[(PtID, percentage)] = {
        "ptid_test": value['PtId_test'],
        "ptid_group": value['ptid_group'],
        "rmse_naive": rmse_naive,
        "rmse_single": rmse_single,
        "rmse_base": rmse_base,
        "rmse_tl": rmse_tl,
        "train_size_single_tl": train_size_single_tl,
        "train_size_base": train_size_base,
        "train_size_naive": train_size_naive
    
    }


In [15]:
import pandas as pd
df = pd.DataFrame.from_dict(dict_results, orient='index').reset_index()
df = df.rename(columns={'level_0': 'PtID', 'level_1': 'ratio'})
df

,PtID,ratio,ptid_test,ptid_group,rmse_naive,rmse_single,rmse_base,rmse_tl,train_size_single_tl,train_size_base,train_size_naive
0,173,70,173,child,1.676099,2.084579,1.519288,1.518595,4081,419677,672
1,32,50,32,child,2.231315,1.998567,1.902960,1.860620,5454,411874,672
2,71,70,71,child,3.144908,3.402323,2.422309,2.397704,2810,387311,672
3,130,50,130,adult,1.704810,1.616094,1.479453,1.478287,4222,407363,672
4,192,80,192,child,2.845309,5.025438,2.609858,2.596030,2873,407946,672
...,...,...,...,...,...,...,...,...,...,...,...
1306,156,100,156,adult,1.801365,1.650195,1.433878,1.430967,4731,392216,672
1307,57,80,57,adult,1.485230,1.417690,1.346232,1.324742,6071,406020,672
1308,155,80,155,adult,1.086988,1.023998,0.963918,0.958983,5955,404264,672
1309,189,100,189,child,3.442447,3.373599,2.755090,2.700814,3391,391538,672


In [18]:
df_merge = df_cgm.merge(df, on=['PtID'])

df_merge

,PtID,Race,Gender,ageAtEnroll,ratio,ptid_test,ptid_group,rmse_naive,rmse_single,rmse_base,rmse_tl,train_size_single_tl,train_size_base,train_size_naive
0,1,black,F,15,70,1,child,2.551910,2.273043,2.007035,2.007232,6226,396464,672
1,1,black,F,15,60,1,child,2.551910,2.338669,2.003175,1.986663,6226,415598,672
2,1,black,F,15,40,1,child,2.551910,2.289941,2.027883,2.000982,6226,405109,672
3,1,black,F,15,80,1,child,2.551910,2.336029,1.992722,1.987042,6226,394897,672
4,1,black,F,15,90,1,child,2.551910,2.265072,2.012234,2.007970,6226,388330,672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,232,black,M,28,70,232,adult,2.784416,2.687245,2.296104,2.224873,2159,403498,672
1307,232,black,M,28,90,232,adult,2.784416,2.901856,2.311735,2.223120,2159,397266,672
1308,232,black,M,28,50,232,adult,2.784416,2.662023,2.302181,2.240261,2159,416585,672
1309,232,black,M,28,100,232,adult,2.784416,2.777348,2.323882,2.235554,2159,390173,672


In [22]:
# save dataframe
file_path_save = f'../results/processed_data/3_1_calculated_ressults/3_1_calculated_results_v6_{group}_v2_test.csv'
df_merge.to_csv(file_path_save)
